In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from datetime import datetime
import re





In [2]:
htmlall = pd.read_html('all_players_1-5-35.html')
_alltbl = htmlall[0]
alltbl = _alltbl.copy()
alltbl.describe()


,Age,WR,Agg,Ant,Bra,Cmp,Cnt,Dec,Fla,Ldr,...,Hea,Lon,L Th,Mar,Fre,Tck,Tec,Pas,Pen,Cro
count,5237.000000,5237.000000,5237.000000,5237.000000,5237.000000,5237.000000,5237.000000,5237.000000,5237.000000,5237.000000,...,5237.000000,5237.000000,5237.000000,5237.000000,5237.000000,5237.000000,5237.000000,5237.000000,5237.000000,5237.000000
mean,28.760932,4801.668131,11.023487,12.784228,11.522055,12.116097,11.612564,12.746038,11.221310,10.075998,...,8.951881,8.221310,5.411304,8.931640,6.404239,8.784610,12.541150,11.760550,6.392973,9.021768
std,4.024958,1684.095293,2.996819,1.758216,2.954671,2.250177,2.822577,1.720558,3.858124,3.150774,...,3.261047,3.117035,3.180586,3.957894,3.468103,4.289091,2.450206,2.251972,3.379692,3.517106
min,18.000000,50.000000,2.000000,4.000000,2.000000,3.000000,1.000000,5.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,1.000000,1.000000
25%,26.000000,3508.000000,9.000000,12.000000,10.000000,11.000000,10.000000,12.000000,8.000000,8.000000,...,7.000000,6.000000,3.000000,6.000000,3.000000,5.000000,11.000000,10.000000,4.000000,7.000000
50%,29.000000,4391.000000,11.000000,13.000000,12.000000,12.000000,12.000000,13.000000,12.000000,10.000000,...,9.000000,9.000000,5.000000,10.000000,6.000000,10.000000,13.000000,12.000000,6.000000,9.000000
75%,32.000000,5880.000000,13.000000,14.000000,13.000000,14.000000,14.000000,14.000000,14.000000,12.000000,...,11.000000,11.000000,7.000000,12.000000,9.000000,12.000000,14.000000,13.000000,9.000000,12.000000
max,38.000000,10000.000000,20.000000,19.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,...,18.000000,17.000000,18.000000,18.000000,17.000000,18.000000,19.000000,18.000000,18.000000,18.000000


In [3]:
alltbl = alltbl.iloc[1:]

In [4]:
def parse_height(height_str):
    match = re.match(r"(\d+)'(\d+)", str(height_str))
    if match:
        feet, inches = map(int, match.groups())
        return feet * 12 + inches
    return None

alltbl['Height'] = alltbl['Height'].apply(parse_height)

In [5]:
alltbl[alltbl.Wage == pd.NA]

,Name,Age,Height,Personality,WR,Transfer Value,Expires,Wage,Last Trans. Fee,Based,...,Tck,Tec,Pas,Pen,Cro,Inf,Av Rat,Apps,Transfer Status,Rec


In [6]:
alltbl['Wage'] = (alltbl['Wage'].str.replace(r"[^\d]", "", regex=True).replace('', pd.NA).astype('Float64'))

In [7]:
def clean_value(value):
    """
    Cleans and converts a transfer value string into a single numeric value (in millions).
    Handles ranges by taking the average.
    """
    if pd.isna(value) or value == 'Not for Sale':
        return np.nan # Return a missing value marker

    # Standardize the string: remove currency symbols and leading/trailing spaces
    # The 'Â' is a common encoding error for '£'
    value_str = str(value).replace('Â£', '').replace('£', '').strip()
    if not value_str:
        return np.nan
        
    # Function to convert M/K suffixes to numbers
    def convert_suffix(val_str):
        val_str = val_str.strip()
        if 'M' in val_str:
            return float(val_str.replace('M', ''))
        if 'K' in val_str:
            return float(val_str.replace('K', '')) / 1000 # Convert K to millions
        return float(val_str)

    # Check if the value is a range
    if '-' in value_str:
        low, high = value_str.split('-')
        low_val = convert_suffix(low)
        high_val = convert_suffix(high)
        return (low_val + high_val) / 2.0
    # Otherwise, it's a single value
    else:
        return convert_suffix(value_str)

alltbl['Transfer Value'] = alltbl['Transfer Value'].apply(clean_value)

In [8]:
def clean_fee(fee):
    """
    Cleans and converts a last transfer fee string into a numeric value in millions.
    Handles 'M' for millions and 'K' for thousands.
    """
    # If the value is already a number or is missing, do nothing.
    if pd.isna(fee) or isinstance(fee, (int, float)):
        return fee

    # Convert to string and standardize it
    fee_str = str(fee).replace('Â£', '').replace('£', '').strip()

    # If the string is empty or a placeholder for no fee, it's a free transfer (0)
    if not fee_str or fee_str in ['-', '- - -', 'Free']:
        return 0.0

    try:
        # Handle values in millions (e.g., '62M')
        if 'M' in fee_str:
            return float(fee_str.replace('M', ''))
        # Handle values in thousands (e.g., '500K')
        elif 'K' in fee_str:
            return float(fee_str.replace('K', '')) / 1000
        # For any other format that fails, return a missing value
        else:
            return np.nan
    except ValueError:
        # If conversion to float fails for any reason, return a missing value
        return np.nan

alltbl['Last Trans. Fee'] = alltbl['Last Trans. Fee'].apply(clean_fee)
alltbl['Transfer Fees Received'] = alltbl['Transfer Fees Received'].apply(clean_fee)

In [9]:
def combine_apps(apps_str):
    """
    Parses a string like '24 (1)' or '22' and returns the sum of the numbers.
    """
    # Return 0 for missing or non-string data
    if pd.isna(apps_str):
        return 0

    # Use a regular expression to find all sequences of digits
    numbers = re.findall(r'\d+', str(apps_str))

    # If numbers are found, convert them to integers and return their sum
    if numbers:
        return sum(int(num) for num in numbers)
    
    # If no numbers are found in the string, return 0
    return 0

alltbl['Total Apps'] = alltbl['Apps'].apply(combine_apps)
print(alltbl[['Name', 'Apps', 'Total Apps']].head())

           Name    Apps  Total Apps
1       Endrick      22          22
2  Emili Artiga  24 (1)          25
3   Vitor Roque      22          22
4  JoÃ£o Veloso  22 (2)          24
5  AarÃ³n Ojeda      22          22


In [10]:
alltbl['Transfer_Status_bool'] = (alltbl['Transfer Status'] != 'Not set').astype(int)

In [11]:
alltbl['Country'] = alltbl['Based'].str.split('(').str[0].str.strip()

In [12]:
display(alltbl)

,Name,Age,Height,Personality,WR,Transfer Value,Expires,Wage,Last Trans. Fee,Based,...,Pen,Cro,Inf,Av Rat,Apps,Transfer Status,Rec,Total Apps,Transfer_Status_bool,Country
1,Endrick,28,68,Resolute,10000,271.0,6/30/2037,575000.0,62.00,Spain (First Division),...,12,15,NaN,7.17,22,Not set,- - -,22,0,Spain
2,Emili Artiga,26,69,Determined,9240,241.5,6/30/2039,325000.0,84.00,England (Premier Division),...,5,15,NaN,7.61,24 (1),Not set,- - -,25,0,England
3,Vitor Roque,29,69,Fairly Professional,10000,240.5,6/30/2037,375000.0,34.50,Spain (First Division),...,14,17,NaN,7.39,22,Not set,- - -,22,0,Spain
4,JoÃ£o Veloso,29,73,Spirited,8635,239.5,6/30/2038,300000.0,63.00,England (Premier Division),...,11,11,NaN,7.33,22 (2),Not set,- - -,24,0,England
5,AarÃ³n Ojeda,24,74,Model Professional,10000,224.5,6/30/2039,375000.0,113.00,England (Premier Division),...,14,14,NaN,7.30,22,Not set,- - -,22,0,England
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5232,Kevin Paredes,31,69,Fairly Determined,5544,NaN,12/31/2036,9500.0,5.25,U.S.A. (MLS),...,9,14,NaN,-,-,Not set,- - -,0,0,U.S.A.
5233,Amine El Ouazzani,33,74,Balanced,4399,NaN,6/30/2036,3900.0,NaN,Morocco (Botola 1),...,12,12,NaN,7.14,25,Not set,- - -,25,0,Morocco
5234,Ransford-Yeboah KÃ¶nigsdÃ¶rffer,33,71,Fairly Ambitious,4784,NaN,6/30/2036,3300.0,NaN,Ghana (Premier Division),...,8,9,NaN,7.47,26 (1),Not set,- - -,27,0,Ghana
5235,Nahuel Noll,31,74,Spirited,4074,NaN,6/30/2036,15500.0,0.75,Spain (Second Division),...,2,1,NaN,6.97,20,Not set,- - -,20,0,Spain


In [13]:
alltbl['Expires'] = pd.to_datetime(alltbl['Expires'], errors='coerce')
alltbl['Begins'] = pd.to_datetime(alltbl['Begins'], errors='coerce')
current_game_date = pd.to_datetime('1/5/2035')

# Calculate the days left on the contract from the IN-GAME date
alltbl['Days Until Expiry'] = (alltbl['Expires'] - current_game_date).dt.days

years_since_signing = (current_game_date - alltbl['Begins']).dt.days / 365.25
alltbl['Age_at_Signing'] = alltbl['Age'] - years_since_signing
alltbl['Years_at_Club'] = (current_game_date - alltbl['Begins']).dt.days / 365.25

# Display the updated calculation
print(alltbl[['Name','Age_at_Signing', 'Years_at_Club' ,'Expires', 'Days Until Expiry']].head())

           Name  Age_at_Signing  Years_at_Club    Expires  Days Until Expiry
1       Endrick       25.607118       2.392882 2037-06-30              907.0
2  Emili Artiga       25.321013       0.678987 2039-06-30             1637.0
3   Vitor Roque       25.613279       3.386721 2037-06-30              907.0
4  JoÃ£o Veloso       26.979466       2.020534 2038-06-30             1272.0
5  AarÃ³n Ojeda       21.459274       2.540726 2039-06-30             1637.0


In [14]:
# Tier 1 Worst (Score = 1)
tier_1 = ['Slack', 'Casual', 'Temperamental', 'Spineless', 'Low Self-Belief', 'Easily Discouraged', 'Low Determination']

# Tier 2 Bad (Score = 2)
tier_2 = ['Fickle', 'Mercenary', 'Unambitious', 'Unsporting', 'Realist']

# Tier 3 Neutral (Score = 3)
tier_3 = ['Balanced', 'Light-Hearted', 'Jovial', 'Very Loyal', 'Devoted', 'Loyal', 'Fairly Loyal', 'Honest', 'Sporting', 'Fairly Sporting']

# Tier 4 Good (Score = 4)
tier_4 = ['Perfectionist', 'Resolute', 'Professional', 'Fairly Professional', 'Iron Willed', 'Resilient', 'Spirited', 'Driven', 'Determined', 'Fairly Determined', 'Charismatic Leader', 'Born Leader', 'Leader', 'Very Ambitious', 'Fairly Ambitious', 'Ambitious']

# Tier 5 Best
tier_5 = ['Model Professional', 'Model Professional']

# Combine them into a single list of lists
personality_tiers = [tier_1, tier_2, tier_3, tier_4, tier_5]

personality_map = {}
# The enumerate function gives us an index (0, 1, 2, ...) and the tier itself
for i, tier in enumerate(personality_tiers):
    # We use i + 1 so scores start from 1 instead of 0
    score = i + 1
    for personality in tier:
        personality_map[personality] = score

alltbl['Personality_Tier'] = alltbl['Personality'].map(personality_map)

In [15]:
alltbl.dtypes

Name                  object
Age                    int64
Height                 int64
Personality           object
WR                     int64
                      ...   
Country               object
Days Until Expiry    float64
Age_at_Signing       float64
Years_at_Club        float64
Personality_Tier     float64
Length: 61, dtype: object